In [105]:
# 根据89611条新闻数据，其中包括78855条新华社新闻，通过多项式分布实现对于抄袭新华社新闻的检测。

In [106]:
import re
import numpy as np
import pandas as pd
import jieba
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from tqdm import tqdm
from pprint import pprint
import os

# 加载停用词
with open('chinese_stopwords.txt','r', encoding='utf-8') as file:
    stopwords=[i[:-1] for i in file.readlines()]
#stopwords = [line.strip() for line in open('chinsesstoptxt.txt',encoding='UTF-8').readlines()]

# 数据加载
news = pd.read_csv('sqlResult.csv',encoding='gb18030')
print(news.shape)
print(news.head(5))

# 处理缺失值
print(news[news.content.isnull()].head(5))
news=news.dropna(subset=['content'])
print(news.shape)

(89611, 7)
      id      author                  source  \
0  89617         NaN  快科技@http://www.kkj.cn/   
1  89616         NaN  快科技@http://www.kkj.cn/   
2  89615         NaN  快科技@http://www.kkj.cn/   
3  89614         NaN                     新华社   
4  89613  胡淑丽_MN7479                   深圳大件事   

                                             content  \
0  此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...   
1  骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...   
2  此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...   
3    这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n   
4  （原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...   

                                             feature  \
0  {"type":"科技","site":"cnbeta","commentNum":"37"...   
1  {"type":"科技","site":"cnbeta","commentNum":"15"...   
2  {"type":"科技","site":"cnbeta","commentNum":"18"...   
3  {"type":"国际新闻","site":"环球","commentNum":"0","j...   
4  {"type":"新闻","site":"网易热门","commentNum":"978",...   

                          

In [108]:
# 分词并去无效词
def split_text(text):
    #return ' '.join([w for w in list(jieba.cut(re.sub('\s|[%s]' % (punctuation),'',text))) if w not in stopwords])
    text = text.replace(' ', '')
    text = text.replace('\n', '')
    text2 = jieba.cut(text.strip())
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result

print(news.iloc[0].content)
print(split_text(news.iloc[0].content))

此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。
MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。
当然，关于MIUI 9的确切信息，我们还是等待官方消息。

 当然 MIUI9 确切 信息 等待 官方消息


In [109]:
#收集并处理content列
if not os.path.exists("corpus.pkl"):
    # 对所有文本进行分词
    corpus=list(map(split_text,[str(i) for i in news.content]))
    print(corpus[0])
    print(len(corpus))
    print(corpus[1])
    # 保存到文件，方便下次调用
    with open('corpus.pkl','wb') as file:
        pickle.dump(corpus, file)
else:
    # 调用上次处理的结果
    with open('corpus.pkl','rb') as file:
        corpus = pickle.load(file)
        print(corpus[0])
        print(len(corpus))
        print(corpus[1])

 当然 MIUI9 确切 信息 等待 官方消息
87054
 当然 高通 骁龙 未来 也许 见到 三星 Exynos 联发科 华为 麒麟 小米 澎湃 进入 Windows10 桌面 平台


In [112]:
# 得到corpus的TF-IDF矩阵
countvectorizer = CountVectorizer(encoding='gb18030',min_df=0.015)
tfidftransformer = TfidfTransformer()

countvector = countvectorizer.fit_transform(corpus)
print(countvector.shape)

tfidf = tfidftransformer.fit_transform(countvector)
print(tfidf[0])

(87054, 884)
  (0, 590)	0.237736608488
  (0, 461)	0.287807002707
  (0, 271)	0.431560057041
  (0, 416)	0.263569509136
  (0, 432)	0.217387276604
  (0, 669)	0.267150299769
  (0, 860)	0.267413014466
  (0, 184)	0.246785583728
  (0, 822)	0.150818545995
  (0, 385)	0.185018693496
  (0, 103)	0.201078563673
  (0, 667)	0.283109136374
  (0, 263)	0.229198530311
  (0, 44)	0.262255632828
  (0, 174)	0.234573238002


In [114]:
# 分类是否为新华社新闻
label=list(map(lambda source: 1 if '新华' in str(source) else 0,news.source))
#print(label)

# 数据集切分
X_train, X_test, y_train, y_test = train_test_split(tfidf.toarray(), label, test_size = 0.3, random_state=42)
clf = MultinomialNB()
clf.fit(X=X_train, y=y_train)

"""
# 进行CV=3折交叉验证
scores=cross_validate(clf, X_train, y_train, scoring=('accuracy','precision','recall','f1'), cv=3, return_train_score=True)
pprint(scores)
"""
y_predict = clf.predict(X_test)
def show_test_reslt(y_true,y_pred):
    print('accuracy:',accuracy_score(y_true,y_pred))
    print('precison:',precision_score(y_true,y_pred))
    print('recall:',recall_score(y_true,y_pred))
    print('f1_score:',f1_score(y_true,y_pred))

show_test_reslt(y_test, y_predict)

accuracy: 0.889726997741
precison: 0.962384953982
recall: 0.914101102242
f1_score: 0.937621832359


In [117]:
# 使用模型检测抄袭新闻
prediction = clf.predict(tfidf.toarray())
labels = np.array(label)
# compare_news_index中有两列：prediction为预测，labels为真实值
compare_news_index = pd.DataFrame({'prediction':prediction,'labels':labels})
# copy_news_index：可能是Copy的新闻（即找到预测为1，但是实际不是“新华社”）
copy_news_index=compare_news_index[(compare_news_index['prediction'] == 1) & (compare_news_index['labels'] == 0)].index
# 实际为新华社的新闻
xinhuashe_news_index=compare_news_index[(compare_news_index['labels'] == 1)].index
print('可能为Copy的新闻索引：', copy_news_index)

可能为Copy的新闻索引： Int64Index([   4,   24,   28,   30,   32,   35,   37,   38,   43,   45,
            ...
            8538, 8539, 8543, 8544, 8546, 8548, 8549, 8551, 8552, 8555],
           dtype='int64', length=2818)


In [118]:
#文章聚类后保存列表
if not os.path.exists("label.pkl"):
    # 使用k-means对文章进行聚类
    from sklearn.preprocessing import Normalizer
    from sklearn.cluster import KMeans
    normalizer = Normalizer()
    scaled_array = normalizer.fit_transform(tfidf.toarray())

    # 使用K-Means, 对全量文档进行聚类
    kmeans = KMeans(n_clusters=25,random_state=42,n_jobs=-1)
    k_labels = kmeans.fit_predict(scaled_array)
    # 保存到文件，方便下次调用
    with open('label.pkl','wb') as file:
        pickle.dump(k_labels, file)
    print(k_labels.shape)
    print(k_labels[0])
else:
    # 调用上次处理的结果
    with open('label.pkl','rb') as file:
        k_labels = pickle.load(file)
        print(k_labels.shape)
        print(k_labels)

(87054,)
[16 16 16 ..., 16 16 23]


In [126]:
#将类别对应编号保存列表
if not os.path.exists("id_class.pkl"):
    # 创建id_class
    id_class = {index:class_ for index, class_ in enumerate(k_labels)}
    # 保存到文件，方便下次调用
    with open('id_class.pkl','wb') as file:
        pickle.dump(id_class, file)
else:
    # 调用上次处理的结果
    with open('id_class.pkl','rb') as file:
        id_class = pickle.load(file)
        print(id_class[0])

16


In [130]:
#按类别排序属于新华社的文章列表
if not os.path.exists("class_id.pkl"):
    from collections import defaultdict
    # 创建你class_id字段，key为classId,value为文档index
    class_id = defaultdict(set)
    for index,class_ in id_class.items():
        # 只统计新华社发布的class_id
        if index in xinhuashe_news_index.tolist():
            class_id[class_].add(index)
    # 保存到文件，方便下次调用
    with open('class_id.pkl','wb') as file:
        pickle.dump(class_id, file)
else:
    # 调用上次处理的结果
    with open('class_id.pkl','rb') as file:
        class_id = pickle.load(file)

In [131]:
# 输出每个类别的 文档个数
count=0
for k in class_id:
    print(count, len(class_id[k]))
    count +=1

# 查找相似文本（使用聚类结果进行filter）
def find_similar_text(cpindex, top=10):
    # 只在新华社发布的文章中查找
    dist_dict={i:cosine_similarity(tfidf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]]}
    # 从大到小进行排序
    return sorted(dist_dict.items(),key=lambda x:x[1][0], reverse=True)[:top]



0 12324
1 10360
2 6919
3 3789
4 3506
5 1851
6 1653
7 1762
8 728
9 2280
10 949
11 1732
12 1603
13 1474
14 1564
15 9749
16 1843
17 874
18 2124
19 2008
20 1358
21 2844
22 2800
23 1254
24 1507


In [133]:
import editdistance
# 指定某篇文章的相似度
#print(copy_news_index)
cpindex = 28 # 在copy_news_index
print('是否在新华社', cpindex in xinhuashe_news_index)
print('是否在copy_news', cpindex in copy_news_index)
#print('28', 28 in xinhuashe_news_index)

#print(cpindex)
similar_list = find_similar_text(cpindex)
print(similar_list)
print('怀疑抄袭:\n', news.iloc[cpindex].content)
# 找一篇相似的原文
similar2 = similar_list[0][0]
print('相似原文:\n', news.iloc[similar2].content)
# 求任意两篇文章的编辑距离 
print('编辑距离:',editdistance.eval(corpus[cpindex], corpus[similar2]))


是否在新华社 False
是否在copy_news True
[(54858, array([[ 0.4790132]])), (54976, array([[ 0.47379451]])), (18495, array([[ 0.46644004]])), (75084, array([[ 0.46600637]])), (82080, array([[ 0.45851198]])), (52291, array([[ 0.45596554]])), (76849, array([[ 0.45554416]])), (86111, array([[ 0.43596508]])), (24206, array([[ 0.43159321]])), (86001, array([[ 0.42976636]]))]
怀疑抄袭:
 中超联赛第13轮比赛已经全部结束，这轮比赛中最让人觉得结果有些出乎意料的，莫过于卡佩罗执教江苏苏宁的主场首秀却以失败告终，在其他球队换帅如换刀的背景下，苏宁却持续陷入低迷，13轮过后只积8分，排名倒数第2，近3年1铁律预示苏宁想保级除非创造奇迹。
就在去年也就是2016赛季，由于中超第21轮被提前，因此中超第21轮比赛结束后也就相当于如今的13轮比赛过后，当时杭州绿城只积9分，山东鲁能由于少赛一场，也积9分，同处降级区，但是鲁能在第13场比赛战平上港，因此13场过后积分应为10分，最终杭州绿城降级，鲁能惊险保级。
再往前推，2015赛季，中超13轮比赛过后，贵州人和与上海申鑫排在倒数第1和第2位，贵州人和积10分，而上海申鑫只有4分，最终这2支球队全部降级，也就是说当时第13轮比赛的降级区球队成为了最终联赛结束后真正降级的2支球队。
然后回到2014赛季，前13轮比赛过后，身处降级区的是哈尔滨毅腾和河南建业两支球队，河南建业已经有10分入账，而哈尔滨毅腾在少赛一场的情况下只积5分，即使算上哈尔滨毅腾第13场比赛能赢，最多也是8分，最终哈尔滨毅腾降级。
从这3年的中超13轮比赛过后的积分榜来看，会发现一个铁律，那就是所有在13轮比赛结束后，还低于10分的球队，最终全部降级，无一幸免。如今江苏苏宁在13轮过后竟然只获得了1场胜利，积分只有8分，接下来的比赛除非创造奇迹打破这三年来的铁律，才可能保级，苏宁从现在开始真的是为保级而战了！（老邱中超球评）

In [134]:
def find_similar_sentence(candidate, raw):
    similist = []
    cl = candidate.strip().split('。')
    ra = raw.strip().split('。')
    for c in cl:
        for r in ra:
            similist.append([c,r,editdistance.eval(c,r)])
    # 最相似的5个句子
    sort=sorted(similist,key=lambda x:x[2])[:5]
    for c,r,ed in sort:
        if c!='' and r!='':
            print('怀疑抄袭句:{0}\n相似原句:{1}\n 编辑距离:{2}\n'.format(c,r,ed))
# 查找copy文章 和第一相似的原文的比对
find_similar_sentence(news.iloc[cpindex].content, news.iloc[similar2].content)


怀疑抄袭句:
从这3年的中超13轮比赛过后的积分榜来看，会发现一个铁律，那就是所有在13轮比赛结束后，还低于10分的球队，最终全部降级，无一幸免
相似原句:\n　　令人稍显意外的是广州恒大，他们直到最后一轮还要确保不输球才能晋级
 编辑距离:61

怀疑抄袭句:
从这3年的中超13轮比赛过后的积分榜来看，会发现一个铁律，那就是所有在13轮比赛结束后，还低于10分的球队，最终全部降级，无一幸免
相似原句:\n　　令很多中国球迷惋惜的是，虽然有三支中超球队晋级，但１６强战就将上演“中国德比”
 编辑距离:62

怀疑抄袭句:
从这3年的中超13轮比赛过后的积分榜来看，会发现一个铁律，那就是所有在13轮比赛结束后，还低于10分的球队，最终全部降级，无一幸免
相似原句:而淘汰赛尚未开打，另一项历史已经出现：苏宁将和上海上港队送出亚冠赛场上的首次“中国德比”
 编辑距离:63

怀疑抄袭句:
从这3年的中超13轮比赛过后的积分榜来看，会发现一个铁律，那就是所有在13轮比赛结束后，还低于10分的球队，最终全部降级，无一幸免
相似原句:这个赛季的苏宁堪称“亚冠球队”，尽管联赛八轮不胜，但在亚冠赛场他们队史首次出线，同时还缔造了一系列中超纪录
 编辑距离:63

怀疑抄袭句:
从这3年的中超13轮比赛过后的积分榜来看，会发现一个铁律，那就是所有在13轮比赛结束后，还低于10分的球队，最终全部降级，无一幸免
相似原句:虽然早早同室操戈，但这也将创造历史：这是中超球队在亚冠上的首度“德比”
 编辑距离:63

